-----------------------------
# DATASET

-----------------------------

**Método de construcción del Dataset de análisis**

1. Búsqueda de todas aquellas rutas que tengan su punto final dentro del área circular que se demarca tomando como radio, el punto más lejano de los puntos que componen las Tbars.
2. Si existe más de una Tbar, determinar a cual de ellas se dirige cada una de las rutas. Esto se realiza mediante la obtención de la distancia euclidia desde el último punto de cada una de las trayectorias hasta el punto runway de cada una de las Tbars, se asume que la menor distancia determina a que Tbar corresponde cada vuelo.
3. Filtrar los vuelos que no crucen la línea marcada por los puntos **right** y **left** de su Tbar correspondiente. 
4. Se toman datos cada **T** muestras para construir el dataset.
5. Filtrado de los puntos de las trayectorias con valores erroneos. Se usa la media y la desviación estándar para determinar cuales de los valores de Latitud y Longitud son erroneos dentro de una trayectoria.
6. Filtrado de vuelos con una cantidad de puntos por debajo de la media/2 de puntos de todos los vuelos.
7. Obtención de los valores escalares y angulos de Velocidad y Aceleración. Tambien se obtiene los valores para la proyección del vector velocidad del viento sobre el de la velocidad del avión.
8. Construcción y guardado del dataset.
-------------------

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import math
from scipy.spatial import distance

from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON

import folium
import randomcolor
from randomcolor import RandomColor

### Parámetros

In [2]:
# Ruta en donde se encuentra el archivos de las Tbars
path_tbar = 'datasets/points.csv'

# Ruta en donde se encuentra el Dataset
path_dataset = 'datasets/budapest_arrivals.csv.zip'

# Ruta en donde se guardara el Dataset generado para análisis
path_save_file = 'datasets/budapest.csv'

# Ruta en donde se salvara el archivo con un vuelo
#path_save_file = 'test/flight.csv'

# Radio en metros. Todos aquellos puntos de las trayectorias  que esten fuera
# de este radio de acción no serán tenidos en cuenta en el dataset.
# Este radio se calculo tomando como centro el punto runway de la Tbar o el promedio
# de las latitudes y longitudes de los puntos runway, si existe más de una Tbar.
radio_filter = 600000

# Intervalo de las muestras. El dataset final es construido con datos cada T muestras
T = 5

# Al final del Notebook se pueden observar a manera de ejemplo algunos vuelos
# del resultado del preprocesamiento del dataset original
# Número de vuelos a graficar
n_flights = 100

## FUNCIONES

In [47]:
%run functions_dataset.py

----------

-------------------------
## 1. Hallar las rutas que se encuentren dentro del área circular demarcada por las Tbar

--------------------------------



Este método encuentra un área circular con centro en el punto RWY hasta el punto mas distante de la Tbar. Si se tiene más de una Tbar se cálcula el punto medio entre los diferentes puntos RWY de las Tbars. De igual forma se cálcula el área desde este punto promedio hasta el punto más lejano de las Tbars.

### 1.1. Carga del archivo que contiene las coordenadas de las Tbars

In [4]:
data_tbars = pd.read_csv(path_tbar)

### 1.2. Convertir las coordenadas de la Tbar a coordenadas Web Mercator

Convierte las coordenadas GPS decimales de las Tbar a coordenadas Web Mercator.

In [5]:
points_tb = points_tbars(data_tbars)
#del data_tbars # Si se desea graficar las trayectorias no se debe borrar esta variable

In [46]:
points_tb

,name,id,lat,lon,wlat,wlon
0,tbar1,center,47.61416666666667,18.983055555555556,6042904.2776,2113184.07814
1,tbar1,right,47.52972222222222,18.869444444444444,6028970.94658,2100536.94711
2,tbar1,left,47.68805555555555,19.08277777777778,6055114.41093,2124285.10514
3,tbar1,rwy,47.445277777777775,19.2575,6015060.03137,2143735.09395
4,tbar2,left,47.21055555555556,19.386388888888888,5976510.07375,2158082.93943
5,tbar2,right,47.36833333333333,19.599166666666665,6002404.05666,2181769.25331
6,tbar2,center,47.29472222222222,19.499722222222225,5990313.62393,2170699.14839
7,tbar2,rwy,47.422777777777775,19.29361111111111,6011357.27359,2147754.96445


### 1.3. Hallar el punto central del área circular a implementar para el filtrado de las rutas

Se toma como punto de cálculo los puntos rwy de cada Tbar

In [6]:
central_p = central_point(points_tb)

### 1.4. Cálcular el radio máximo entre el punto central al punto más lejano de las Tbar

Se mide la distancia euclidea entre el punto central a cada uno de los puntos 'left' and 'right' de las Tbar y se selecciona aquel que sea el más distante.

In [7]:
radio = radio_max(central_p, points_tb)

### 1.5. Dibujar el punto, la circunferencia y las Tbar para verificar su posición

(OPCIONAL)

In [8]:
lat_airport = 47.436998252 
lon_airport = 19.257165638

result_cond, coord = geographic_to_web_mercator(lat_airport, lon_airport)

zoom = 50000

fig = figure(tools='pan, wheel_zoom, reset', x_range=(coord[1] - zoom, coord[1] + zoom),
             y_range=(coord[0] - zoom, coord[0] + zoom))


# Punto central
fig.circle(central_p['wlon'].values[0], central_p['wlat'].values[0], size=10, color="red", alpha=0.5)
# Circunferencia
fig.arc(x = central_p['wlon'].values[0], y = central_p['wlat'].values[0], start_angle=0,
        end_angle=2*math.pi, radius = radio, direction='clock', color='red')

fig.axis.visible = False
fig.add_tile(CARTODBPOSITRON)


names_tbars = pd.unique(points_tb['name'])
for name_tbar in names_tbars:
    Tbar = points_tb[points_tb['name'] == name_tbar]
    ids_Tbar = pd.unique(Tbar['id'])
    for id_Tbar in ids_Tbar:
        wlon = Tbar[Tbar['id'] == id_Tbar]['wlon'].astype(float).values[0]
        wlat = Tbar[Tbar['id'] == id_Tbar]['wlat'].astype(float).values[0]
        fig.circle(wlon, wlat, size=5, color="blue", alpha=0.5)   

#show(fig)

### 1.6. Leer el dataset de los vuelos

In [9]:
# Para un vuelo
#data = pd.read_csv('test/flight_id_301.csv', index_col=0)

In [10]:
#data = pd.read_csv('../datasets/budapest_arrivals.csv.zip', compression='zip', nrows=1000000)
data = pd.read_csv(path_dataset, compression='zip')

### 1.7. Convertir las coordenadas GPS del Dataset a coordenadas Web Mercator y Filtrado de puntos

Se toman todos los datos de Latitud y Longitud se transforman a Web Mercator, se crea un dataframe con ellos y se concatenan con el dataframe data que contiene el dataset de los vuelos.

Los puntos de Lat y Lon que esten por fuera de los rangos de lat y lon son borrados.

In [11]:
data = GPS_to_WebMercator(data)

** Filtrar puntos**

Se establece un radio de 600 Km. Cualquier punto que se encuentre por encima de este rango es eliminado del dataset.

In [12]:
data = filter_points_radius(data, central_p, radio_filter)
data = data.reset_index(drop=True)

### 1.8 Hallar el punto final de cada uno de los vuelos

In [13]:
flights_Fp = search_Fpoint(data)

### 1.9. Hallar las rutas que tengan su punto final dentro del área de la circunferencia

In [14]:
# Vuelos que se encuentran dentro del área circular

flights_valid = validate_Fp(radio, central_p, flights_Fp)
flights_Fp = flights_Fp[flights_valid]

### 1.10. Construir nuevo dataframe con los datos filtrados

In [15]:
ids = flights_Fp['ID'].values
data = data[data.ID.isin(ids)].copy(deep=False)
data = data.reset_index(drop=True)

-----------------------------
## 2. Hallar a cuál punto rwy esta más cerca el último punto de cada ruta

------------------------------------------------

Esto se hace bajo la suposición de que ese último punto indica a que Tbar se dirige. Esto se hace midiendo la distancia Euclidea del último punto tomado de cada ruta a los puntos de las Tbars.

In [16]:
Tbar_direction = select_Tbar(flights_Fp, points_tb)

-----------------
## 3. Agregar el campo Tbar al dataset que contiene los vuelos

-----------------

In [17]:
id_flights = data['ID'].drop_duplicates().values
Tbar = []

for id_flight in id_flights:
    data_aux = len(data[data['ID'] == id_flight])
    name_Tbar = Tbar_direction[Tbar_direction['ID'] == id_flight]['Tbar']
    Tbar = Tbar + ([name_Tbar.values[0] for x in range(data_aux)])

Tbar_df = pd.DataFrame(np.asarray(Tbar), columns=list(['Tbar']))
Tbar_df = Tbar_df.reset_index(drop=True)
data = pd.concat([data, Tbar_df], axis=1)

----------------
## 4. Determinar cuáles de las rutas pasan por la línea de cada una de las Tbar correspondiente
----------------

Todos aquellos puntos que esten por encima de la recta de las Tbars, al evaluarlos en la ecuación de la recta, darán negativo si se encuentran encima de ella o positivo en caso contrario.

La variable **points_tb** contiene la información de las Tbars.

### 4.1. Obtener los vuelos que cruzan la línea de la Tbar

- Se cálcula el punto siguiente para los vuelos que cruzan la línea de la Tbar.
- Del punto se obtiene su posición (Lat y Lon en metros) y su TimeStamp.

In [18]:
flight_bool = []
point_data = pd.DataFrame()

names_Tbar = points_tb['name'].drop_duplicates().values

for name_Tbar in names_Tbar:
    
    #print(name_Tbar)
    
    data_flightsTbar = data[data['Tbar'] == name_Tbar]
    id_flights = data_flightsTbar['ID'].drop_duplicates().values
    
    
    #print(len(id_flights))
    
    
    for id_flight in id_flights:
        
        x = data_flightsTbar[data_flightsTbar['ID'] == id_flight]['LONGITUDE_meters'].values
        y = data_flightsTbar[data_flightsTbar['ID'] == id_flight]['LATITUDE_meters'].values
        
        eq = eq_line(points_tb, name_Tbar, x, y)
        
        if len(eq) > 0 :
            eq = (np.diff(np.sign(eq)) != 0)*1   # pone todo a cero y un 1 donde se produce el cambio de indice
            ind = np.argmax(eq)  # Retorna el indice donde se encuentra el punto mas cercano a la línea
                                 # Este indice es con respecto al vector que se extrajo para cada vuelo
            data_flight = data[data['ID'] == id_flight][['ID', 'LATITUDE_meters', 'LONGITUDE_meters', 'TIMESTAMP', 'Tbar']]
            
            test = data_flight.iloc[ind]
            test = test.to_frame().T
            
            point_data = pd.concat([point_data, test], axis=0)
            
            # El indice tiene que ser mayor a cero            
            if (ind > 0):
                flight_bool.append(True)
            else:
                flight_bool.append(False)
            
point_data = point_data.reset_index(drop=True)
point_data['ID'] = point_data['ID'].astype('int64')
point_data = point_data[flight_bool]
point_data = point_data.reset_index(drop=True)

### 4.2. Descartar del Dataset los vuelos que no cruzan por la línea de la Tbar

In [19]:
ids = point_data['ID'].values
data = data[data.ID.isin(ids)].copy(deep=False)
data = data.reset_index(drop=True)

## 5. Seleccionar los datos cada T muestras

- Cada muestra de tiempo (TIMESTAMP) esta en microsegundos.
- Cada muestras es 1 segundo
- Si se toman cada T muestras se supone que el dato cambia T seg
- Al eliminar puntos con posiciones erroneas se perdieron muestras

In [20]:
data = data.iloc[list( i for i in range(0, len(data), T))]
data = data.reset_index(drop=True)

## 6. Filtrar puntos de la trayectoria con valores erroneos

Esto se hace mediante el filtrado de los valores que sean mayores a la media + factor*desviación_estándar.

In [21]:
ids = np.unique(data['ID'].values)

In [22]:
cont_flights = 0
data_new = pd.DataFrame()
factor = 2

for id_f in ids:
    data_id = data[data['ID'] == id_f]
    
    lon_mean = data_id.LONGITUDE_meters.mean()
    lon_std = data_id.LONGITUDE_meters.std()    
    
    for point_lon in data_id.LONGITUDE_meters:
        if point_lon > (lon_mean + factor*lon_std):
            data_id = data_id.drop(data_id[data_id.LONGITUDE_meters == point_lon].index )
        if point_lon < (lon_mean - factor*lon_std):
            data_id = data_id.drop(data_id[data_id.LONGITUDE_meters == point_lon].index )
            
    data_new = pd.concat([data_new, data_id])
    
data_new = data_new.reset_index(drop=True)

In [23]:
data = data_new.copy()
del data_new

## 7. Filtrar vuelos que tengan un número de puntos por debajo de la media

In [24]:
mean_flights_points = int(np.trunc(np.unique(((data.groupby('ID').count()).mean()).values)))

In [25]:
for id_f in ids:
    long = len(data[data['ID'] == id_f])
    if long < mean_flights_points:
        data = data.drop(data[data['ID'] == id_f].index)
        cont_flights += 1

In [26]:
data = data.reset_index(drop=True)

## 8. Cálcular la distancia de cada muestra a la recta de la Tbar

In [45]:
# para cada ID sacar la distacia euclidea entre el punto y su punto de intercepcion

id_flights =  data['ID'].drop_duplicates().values

distTbar = pd.DataFrame()
dist = 0
for id_flight in id_flights:
    
    x1 = data[data['ID'] == id_flight]['LONGITUDE_meters'].values
    y1 = data[data['ID'] == id_flight]['LATITUDE_meters'].values
    
    
    x2 = point_data[point_data['ID'] == id_flight]['LONGITUDE_meters'].values[0]
    y2 = point_data[point_data['ID'] == id_flight]['LATITUDE_meters'].values[0]
    
     
    p1 = np.stack((x1, y1), axis=-1)
    p2 = np.stack((x2, y2), axis=-1)
    p2 = p2.reshape(1,2)
    dist = distance.cdist(p1, p2, 'euclidean')
    
    distAux = pd.DataFrame(np.asarray(dist), columns=list(['dist_Tbar']))            
    distTbar = pd.concat([distTbar, distAux], axis=0)
            
distTbar = distTbar.reset_index(drop=True)

## 9. Concatenar el Dataframe dist_Tbar con el de los datos de vuelo

In [28]:
data = pd.concat([data, distTbar], axis=1)
data = data.reset_index(drop=True)

## 10. Cálcular el vector velocidad y su ángulo para cada punto

In [29]:
Vx = data['VELOCITY_EW'].values
Vy = data['VELOCITY_NS'].values
V = (np.sqrt(np.square(Vx) + np.square(Vy)))
angleV = calc_angle(Vx, Vy)
Vel = np.stack((V, angleV), axis=-1)    
Vel = pd.DataFrame(Vel, columns=list(['Velocity', 'Angle_V']))
Vel = Vel.reset_index(drop=True)

## 11. Concatenar el Dataframe Vel con el de Datos de Vuelo

In [30]:
data = pd.concat([data, Vel], axis=1)
data = data.reset_index(drop=True)

## 12. Cálcular el vector de la aceleración y el ángulo para cada punto

In [31]:
Ax = data['ACCELERATION_EW'].values
Ay = data['ACCELERATION_NS'].values
A = (np.sqrt(np.square(Ax) + np.square(Ay)))
angleA = calc_angle(Ax, Ay)
Acc = np.stack((A, angleA), axis=-1)    
Acc = pd.DataFrame(Acc, columns=list(['Acceleration', 'Angle_A']))
Acc = Acc.reset_index(drop=True)

## 13. Concatenar el Dataframe Acc con el de Datos de Vuelo

In [32]:
data = pd.concat([data, Acc], axis=1)
data = data.reset_index(drop=True)

## 14. Cálcular el vector del Viento y su ángulo para cada punto

In [33]:
Wx = data['WIND_EW'].values
Wy = data['WIND_NS'].values
W = (np.sqrt(np.square(Wx) + np.square(Wy)))
angleW = calc_angle(Wx, Wy)
Wcc = np.stack((W, angleW), axis=-1)    
Wcc = pd.DataFrame(Wcc, columns=list(['Wind', 'Angle_W']))
Wcc = Wcc.reset_index(drop=True)

## 15. Velocidad del avión relacionada con la velocidad del viento

In [34]:
V = data[['VELOCITY_EW', 'VELOCITY_NS']].values
W = data[['WIND_EW', 'WIND_NS']].values
Vpv = []
for val in range(len(V)):
    Vpv.append(ortho_projection(W[val],V[val]))

In [35]:
#Vp = pd.DataFrame(Vpv, columns=list(['Velocity_p', 'Angle_p']))
Vp = pd.DataFrame(Vpv, columns=list(['Velocity_p']))
data = pd.concat([data, Vp], axis=1)
data = data.reset_index(drop=True)

## 16. Concatenar el Dataframe Wcc con el de Datos de Vuelo

In [36]:
data = pd.concat([data, Wcc], axis=1)
data = data.reset_index(drop=True)

## 17. Obtener el delta de tiempo desde cada punto T seg hasta el punto P donde se cruza con la Tbar 

- La variable **point_data** contiene el Dataframe con los puntos donde se cruza cada ruta con la Tbar
- La variable **data** Contiene el Dataframe con los datos de los vuelos cada T seg-

In [37]:
# para cada ID sacar el delta entre el tiempo de ese punto y su punto de intercepcion
id_flights =  data['ID'].drop_duplicates().values

point_target = []
point_limit = []

for id_flight in id_flights:
    
    time = data[data['ID'] == id_flight]['TIMESTAMP'].values
    time_base = point_data[point_data['ID'] == id_flight]['TIMESTAMP'].values
    
    points_valid = np.less(time, time_base)
    subtract = (np.subtract(time,time_base)/1000000) * -1
    
    point_limit = point_limit + points_valid.tolist()
    point_target = point_target + subtract.tolist()

target = pd.DataFrame(np.asarray(point_target), columns=list(['Target_sec']))        


## 18. Concatenar el Dataframe Target con el de Datos de los vuelos

In [38]:
data = pd.concat([data, target], axis=1)
data = data.reset_index(drop=True)

## 19. Filtrar los puntos de los vuelos que sobrepasan la recta de la Tbar

In [39]:
data = data[point_limit]
data = data.reset_index(drop=True)

## 21. Convertir la velocidad de knots a m/s

In [40]:
data['Velocity'] = convert_knot_ms(data['Velocity'].values)

## 20. Dataset

In [41]:
dataset = data[['ID', 
                'CALLSIGN', 
                'AC_REAL_MODEL',
                'Tbar',
                'LATITUDE_meters', 
                'LONGITUDE_meters', 
                'VERTICAL_RATE', 
                'VERTICAL_ACCELERATION', 
                'Velocity', 
                'Angle_V', 
                'Acceleration', 
                'Angle_A', 
                'dist_Tbar',
                'Velocity_p',
                'Target_sec']]

In [42]:
dataset.to_csv(path_save_file)

## 21. Grafica de vuelos

OPCIONAL

In [43]:
# Solo para cuando se preprocesa el dataset con multiples vuelos
#El numero de vuelos a leer debe ser menos al numero de vuelos en el dataset

data = read_flights(data, n_flights)
data = data.reset_index(drop=True)

In [44]:
# folium.Map: Budapest Airport [47.434327, 19.258816]
coord_bud_airp = [47.5657773025, 19.0302945977]
map_budap = folium.Map(location=coord_bud_airp, zoom_start=6)

# Marker at Budapest Airport
#folium.Marker(location=coord_bud_airp, popup="Budapest Airport Coordenates"+str(coord)).add_to(map_budap)

# Random Color
rand_color = randomcolor.RandomColor()

# Calculate number of flights: max_id 
max_id = np.unique(data.ID.values)

for id in max_id:
    # Latitude and Longitude for each flight ID: points
    points = []
    points = data[data['ID'] == id][['LATITUDE', 'LONGITUDE']].values.tolist()
    
    # folium.PolyLine
    map_budap_poly = folium.PolyLine(locations=points,
                                     color=rand_color.generate(), 
                                     weight=1.5, 
                                     opacity=0.8).add_to(map_budap)

# TBAR1
# tbar1 data: tbar1
tbar1 = data_tbars.values[:,2:4][data_tbars['name']=='tbar1'].tolist()

tbar1_coord = []
for x in range(0, len(tbar1)):
    #Latitude 
    tmp = tbar1[x][0][0:-1]
    lat = int(tmp[0:2]) + int(tmp[2:4])/60 + int(tmp[4:6])/3600
    if(tmp[-1] == 'S'):
        lat = -1 * lat    
    
    #Longitude
    tmp = tbar1[x][1][0:-1].strip()
    if(tmp[0] == '0'):
        tmp = tmp[1:]
    lon = int(tmp[0:2]) + int(tmp[2:4])/60 + int(tmp[4:6])/3600
    if(tmp[-1] == 'W'):
        lon = -1 * lat
    
    #tbar1_coord: lat, lon
    tbar1_coord.append([lat, lon])

# Dotting tbar1 coordenates
for coord in tbar1_coord: 
    folium.Circle(location=coord,
                popup=str(coord),
                radius=200,
                color='crimson',
                fill=True,
                fill_color='crimson'
                ).add_to(map_budap)

# Drawing tbar1
left_to_right = [tbar1_coord[0], tbar1_coord[3]]
center_to_rwy = [tbar1_coord[1], tbar1_coord[2]]

folium.PolyLine(
    locations=left_to_right,
    weight=2,
    color='black'
).add_to(map_budap)


folium.PolyLine(
    locations=center_to_rwy,
    weight=2,
    color='black'
).add_to(map_budap)

# TBAR2
# tbar2 data: tbar2
tbar2 = data_tbars.values[:,2:4][data_tbars['name']=='tbar2'].tolist()

tbar2_coord = []
for x in range(0, len(tbar2)):
    #Latitude 
    tmp = tbar2[x][0][0:-1]
    lat = int(tmp[0:2]) + int(tmp[2:4])/60 + int(tmp[4:6])/3600
    if(tmp[-1] == 'S'):
        lat = -1 * lat    
    
    #Longitude
    tmp = tbar2[x][1][0:-1].strip()
    if(tmp[0] == '0'):
        tmp = tmp[1:]
    lon = int(tmp[0:2]) + int(tmp[2:4])/60 + int(tmp[4:6])/3600
    if(tmp[-1] == 'W'):
        lon = -1 * lat
    
    #tbar2_coord: lat, lon
    tbar2_coord.append([lat, lon])

# Dotting tbar2 coordenates
for coord in tbar2_coord: 
    folium.Circle(location=coord,
                popup=str(coord),
                radius=200,
                color='crimson',
                fill=True,
                fill_color='crimson'
                ).add_to(map_budap)

# Drawing tbar2
left_to_right = [tbar2_coord[0], tbar2_coord[1]]
center_to_rwy = [tbar2_coord[2], tbar2_coord[3]]

folium.PolyLine(
    locations=left_to_right,
    weight=2,
    color='blue'
).add_to(map_budap)


folium.PolyLine(
    locations=center_to_rwy,
    weight=2,
    color='blue'
).add_to(map_budap)

map_budap